In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [56]:
orders_file = pd.read_parquet('Data/orders.parquet')
orders_file.head()


In [ ]:
orders_file.columns

In [58]:
# X = orders_file['currency'].unique().tolist()
# print(X)
# ['Euro', 'Australian Dollar', 'Brazilian Real', 'US Dollar', 'Danish Krone', 'Saudi Riyal', 'Mexican Peso', 'Pound Sterling', 'Zloty', 
#  'Norwegian Krone', 'Canadian Dollar', 'United Arab Emirates dirham', 'Swedish Krona', 'Chilean Peso', 'Peso Uruguayo', 'Nuevo Sol Peru', 
#  'South Korean Won', 'Malaysian Ringgit', 'Argentine Peso', 'Thai Baht', 'Czech Koruna', 'Colombian Peso Colombia', 'Kuwaiti Dinar', 'Swiss Franc', 
#  'Hryvnia Ukraine', 'South African Rand', 'Japanese yen', 'Jordanian Dinar', 'Bahraini Dinar', 'New Zealand Dollar', 'Indian Rupee', 'Egyptian Pound', 
#  'Bulgarian Lev', 'Rupiah Indonesia', 'Turkish Lira', 'Qatari Rial', 'Singapore Dollar', 'Hong Kong Dollar', 'Philippine Peso', 'New Taiwan Dollar', 
#  'Rial Omani Oman', 'Forint', 'Yuan Renminbi', 
#  'Vietnamese dong', 'Iceland Krona', 'Tenge Kazakhstan', 'Uzbekistan Som']



currency_name_to_code = {
    'Euro': 'EUR',
    'Australian Dollar': 'AUD',
    'Brazilian Real': 'BRL',
    'US Dollar': 'USD',
    'Danish Krone': 'DKK',
    'Saudi Riyal': 'SAR',
    'Mexican Peso': 'MXN',
    'Pound Sterling': 'GBP',
    'Zloty': 'PLN',
    'Norwegian Krone': 'NOK',
    'Canadian Dollar': 'CAD',
    'United Arab Emirates dirham': 'AED',
    'Swedish Krona': 'SEK',
    'Chilean Peso': 'CLP',
    'Peso Uruguayo': 'UYU',
    'Nuevo Sol Peru': 'PEN',
    'South Korean Won': 'KRW',
    'Malaysian Ringgit': 'MYR',
    'Argentine Peso': 'ARS',
    'Thai Baht': 'THB',
    'Czech Koruna': 'CZK',
    'Colombian Peso Colombia': 'COP',
    'Kuwaiti Dinar': 'KWD',
    'Swiss Franc': 'CHF',
    'Hryvnia Ukraine': 'UAH',
    'South African Rand': 'ZAR',
    'Japanese yen': 'JPY',
    'Jordanian Dinar': 'JOD',
    'Bahraini Dinar': 'BHD',
    'New Zealand Dollar': 'NZD',
    'Indian Rupee': 'INR',
    'Egyptian Pound': 'EGP',
    'Bulgarian Lev': 'BGN',
    'Rupiah Indonesia': 'IDR',
    'Turkish Lira': 'TRY',
    'Qatari Rial': 'QAR',
    'Singapore Dollar': 'SGD',
    'Hong Kong Dollar': 'HKD',
    'Philippine Peso': 'PHP',
    'New Taiwan Dollar': 'TWD',
    'Rial Omani Oman': 'OMR',
    'Forint': 'HUF',
    'Yuan Renminbi': 'CNY',
    'Vietnamese dong': 'VND',
    'Iceland Krona': 'ISK',
    'Tenge Kazakhstan': 'KZT',
    'Uzbekistan Som': 'UZS'
}
#Static exchange rate, in actual analysis it has to be updated so it uses API, easy-exchange-rates and forex-python has been tried so far, 
#Read their documentationss to see why the exchange has problems. 
exchange_rates_to_usd = {
    'USD': 1,
    'EUR': 1.08,
    'AUD': 0.64,
    'BRL': 0.19,
    'DKK': 0.14,
    'SAR': 0.27,
    'MXN': 0.052,
    'GBP': 1.37,
    'PLN': 0.26,
    'NOK': 0.11,
    'CAD': 0.75,
    'AED': 0.27,
    'SEK': 0.095,
    'CLP': 0.0012,
    'UYU': 0.023,
    'PEN': 0.27,
    'KRW': 0.00076,
    'MYR': 0.23,
    'ARS': 0.010,
    'THB': 0.032,
    'CZK': 0.048,
    'COP': 0.00026,
    'KWD': 3.28,
    'CHF': 1.09,
    'UAH': 0.036,
    'ZAR': 0.055,
    'JPY': 0.0074,
    'JOD': 1.41,
    'BHD': 2.65,
    'NZD': 0.63,
    'INR': 0.013,
    'EGP': 0.032,
    'BGN': 0.58,
    'IDR': 0.000066,
    'TRY': 0.054,
    'QAR': 0.27,
    'SGD': 0.74,
    'HKD': 0.13,
    'PHP': 0.018,
    'TWD': 0.032,
    'OMR': 2.60,
    'HUF': 0.0031,
    'CNY': 0.15,
    'VND': 0.000043,
    'ISK': 0.0075,
    'KZT': 0.0023,
    'UZS': 0.000091
}



def convert_prices_to_usd(df):
    
    df['currency_code'] = df['currency'].map(currency_name_to_code)

    currency_index = df.columns.get_loc('currency')  
    df.insert(currency_index + 1, 'currency_code', df.pop('currency_code'))

    # Check for unmapped currencies
    unmapped = df[df['currency_code'].isna()]['currency'].unique()
    if len(unmapped) > 0:
        print("Unmapped currencies:", unmapped)

    
    df['Order_Amount_in_usd'] = df.apply(
        lambda row: row['Order_Amount'] * exchange_rates_to_usd.get(row['currency_code'], 0) if row['currency_code'] in exchange_rates_to_usd else 0,
        axis=1
    )
    
    price_index = df.columns.get_loc('Order_Amount')
    df.insert(price_index + 1, 'Order_Amount_in_usd', df.pop('Order_Amount_in_usd'))

    df['Revenue_in_usd'] = df.apply(
        lambda row: row['Revenue'] * exchange_rates_to_usd.get(row['currency_code'], 0) if row['currency_code'] in exchange_rates_to_usd else 0,
        axis=1
    )
    
    price_index = df.columns.get_loc('Revenue')
    df.insert(price_index + 1, 'Revenue_in_usd', df.pop('Revenue_in_usd'))

    return df


orders_file = convert_prices_to_usd(orders_file)

In [ ]:
orders_file['Is_Canceled'].value_counts()

In [ ]:
counts = orders_file['Is_Canceled'].value_counts()
canceled_percentage = counts.get(1,0) / counts.get(0,1) * 100

print(f"{canceled_percentage:.2f}% of orders are canceled")

In [ ]:
counts = orders_file['Is_Changed'].value_counts()
changed_percentage = counts.get(1,0) / counts.get(0,1) * 100

print(f"{changed_percentage:.2f}% of orders are changed")

In [ ]:
orders_file.loc[orders_file['Is_Canceled'] == 1, 'cancel_reason'].unique()

In [ ]:
canceled_value_counts = orders_file.loc[orders_file['Is_Canceled'] == 1, 'cancel_reason'].value_counts()

threshold = canceled_value_counts.sum() * 0.02

canceled_counts = canceled_value_counts[canceled_value_counts > threshold]

other_count = orders_file[(orders_file['Is_Canceled'] == 1) & ~orders_file['cancel_reason'].isin(canceled_counts.index)]['cancel_reason'].count()
canceled_counts['Other reasons'] = other_count
canceled_counts['Other reasons']

plt.figure(figsize=(8, 8))
plt.pie(canceled_counts, labels=canceled_counts.index, autopct='%1.1f%%', startangle=90)
plt.title("Distribution of cancelation reason")
plt.axis('equal')  
plt.show()



In [ ]:
orders_file.columns

In [ ]:
changed_value_counts = orders_file.loc[orders_file['Is_Changed'] == 1, 'change_reason'].value_counts()

threshold = changed_value_counts.sum() * 0.02

changed_counts = changed_value_counts[changed_value_counts > threshold]

other_count = orders_file[(orders_file['Is_Changed'] == 1) & ~orders_file['change_reason'].isin(changed_counts.index)]['change_reason'].count()
changed_counts['Other reasons'] = other_count
changed_counts['Other reasons']

plt.figure(figsize=(8, 8))
plt.pie(changed_counts, labels=changed_counts.index, autopct='%1.1f%%', startangle=90)
plt.title("Distribution of changes on Orders")
plt.axis('equal')  
plt.show()



### Which country is the on top of the origin country `Origin_Country`?

In [ ]:
top_tourists = orders_file['Origin_Country'].value_counts().head(1)

origin_name = top_tourists.index[0]
origin_count = top_tourists.values[0]


print(f"Most travels start from: {origin_name} with {origin_count} number")

### Which country is The most attractive destination

In [ ]:
most_attractive = orders_file['Destination_Country'].value_counts().head(1)

destination_name = most_attractive.index[0]
destination_count = most_attractive.values[0]


print(f"The most attractive destination is: {destination_name} with {destination_count} visits")

In [ ]:
 
average_price_per_country = orders_file.groupby('Site_Country')['Order_Amount_in_usd'].mean()

highest_order_country = average_price_per_country.idxmax()
highest_average_order = average_price_per_country.max()

average_revenue_per_country = orders_file.groupby('Site_Country')['Revenue_in_usd'].mean()

highest_revenue_country = average_revenue_per_country.idxmax()
highest_average_revenue = average_revenue_per_country.max()

print(f"The country with the highest average amount of payment is: {highest_order_country} with  {highest_average_order:.2f}USD")
print(f"The country with the highest average revenue is: {highest_revenue_country} with an average revenue of {highest_average_revenue:.2f}USD")


top_10_order_countries = average_price_per_country.sort_values(ascending=False).head(10)

plt.figure(figsize=(10, 6))
top_10_order_countries.plot(kind='bar', color='skyblue')

plt.title('Top 10 Countries with the Highest order amount', fontsize=16)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Order amount (USD)', fontsize=12)


plt.xticks(rotation=45, ha='right')
plt.tight_layout()  
plt.show()


In [ ]:
orders_file['Brand'].value_counts()

In [ ]:

orders_file['Customer_Group_Type'].value_counts()

In [ ]:
orders_file['client_entry_type'].value_counts()

In [ ]:

orders_file['Device'].value_counts()

In [ ]:

orders_file['Journey_Type_ID'].value_counts()

In [ ]:

plt.figure(figsize=(10, 6))
sns.countplot(x='Brand', hue='Customer_Group_Type', data=orders_file)
plt.title('Count of Customer Group Types by Brand')
plt.xlabel('Brand')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


In [78]:
bins = [0, 100, 500, 1000, 5000, float('inf')]  
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

orders_file['Order_Value_Category'] = pd.cut(
    orders_file['Order_Amount_in_usd'], 
    bins=bins, 
    labels=labels, 
    include_lowest=True
)

cancellation_prob = orders_file.groupby('Order_Value_Category').agg(
    Cancellation_Probability=('Is_Canceled', 'mean')
).reset_index()

print(orders_file[['Order_Value_Category', 'Order_Amount_in_usd']])
print(cancellation_prob)


plt.figure(figsize=(8, 6))
plt.bar(cancellation_prob['Order_Value_Category'], cancellation_prob['Cancellation_Probability'], color='skyblue')
plt.xlabel('Order Value Category')
plt.ylabel('Cancellation Probability')
plt.title('Cancellation Probability by Order Value')
plt.xticks(rotation=45)
plt.show()


In [ ]:
origin_dest_counts = orders_file.groupby(['Origin_Country', 'Destination_Country','Site_Country']).size().reset_index(name='Count')

sorted_origin_dest = origin_dest_counts.sort_values(by='Count', ascending=False)

top_10_origin_dest = sorted_origin_dest.head(10)

print(top_10_origin_dest)

plt.figure(figsize=(10, 6))
sns.barplot(data=top_10_origin_dest, x='Count', y='Origin_Country', hue='Destination_Country')
plt.title('Top 10 Most Common Origin-Destination Pairs')
plt.xlabel('Count')
plt.ylabel('Origin Country')
plt.show()

In [ ]:
filtered_orders = orders_file[orders_file['Origin_Country'] != orders_file['Destination_Country']]

origin_dest_counts = filtered_orders.groupby(['Origin_Country', 'Destination_Country', 'Site_Country']).size().reset_index(name='Count')

sorted_origin_dest = origin_dest_counts.sort_values(by='Count', ascending=False)

top_10_origin_dest = sorted_origin_dest.head(10)

print(top_10_origin_dest)

plt.figure(figsize=(10, 6))
sns.barplot(data=top_10_origin_dest, x='Count', y='Origin_Country', hue='Destination_Country')
plt.title('Top 10 Most Common Origin-Destination Pairs (Excluding Same-Origin Destinations)')
plt.xlabel('Count')
plt.ylabel('Origin Country')
plt.show()


In [ ]:
orders_file['Journey_Type_ID'].unique()

In [ ]:
uniq_per_id = orders_file.groupby('order_id')['PNR'].nunique()

ids_with_more_name = uniq_per_id[uniq_per_id>1]

if ids_with_more_name.empty:
    print("Each id belongs to only one name.")
else:
    print('The following ids are associated with multiple names:')
    print(ids_with_more_name)

In [84]:
df = orders_file[orders_file['Order_Amount_in_usd'] > 0].copy()
df['ratio_revenue_to_total'] = (df['Revenue_in_usd'] / df['Order_Amount_in_usd']) * 100
average_ratio = df['ratio_revenue_to_total'].mean()

print(f"{average_ratio:.2f}%")

In [ ]:
destination_analysis = df.groupby('Destination_Country')['ratio_revenue_to_total'].mean().reset_index()
destination_analysis = destination_analysis.sort_values(by='ratio_revenue_to_total', ascending=False)

origin_analysis = df.groupby('Origin_Country')['ratio_revenue_to_total'].mean().reset_index()
origin_analysis = origin_analysis.sort_values(by='ratio_revenue_to_total', ascending=False)

customer_group_analysis = df.groupby('Customer_Group_Type')['ratio_revenue_to_total'].mean().reset_index()
customer_group_analysis = customer_group_analysis.sort_values(by='ratio_revenue_to_total', ascending=False)

print("Top Destination Countries by Revenue-to-Total Ratio:")
print(destination_analysis.head())

print("\nTop Origin Countries by Revenue-to-Total Ratio:")
print(origin_analysis.head())

print("\nTop Customer Group Types by Revenue-to-Total Ratio:")
print(customer_group_analysis.head())

In [ ]:

sns.set(style="whitegrid")

plt.figure(figsize=(12, 6))
sns.barplot(data=destination_analysis.head(10), x='ratio_revenue_to_total', y='Destination_Country', palette='viridis')
plt.title('Top 10 Destination Countries by Revenue-to-Total Ratio', fontsize=16)
plt.xlabel('Revenue-to-Total Ratio', fontsize=12)
plt.ylabel('Destination Country', fontsize=12)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.barplot(data=origin_analysis.head(10), x='ratio_revenue_to_total', y='Origin_Country', palette='plasma')
plt.title('Top 10 Origin Countries by Revenue-to-Total Ratio', fontsize=16)
plt.xlabel('Revenue-to-Total Ratio', fontsize=12)
plt.ylabel('Origin Country', fontsize=12)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.barplot(data=customer_group_analysis.head(10), x='ratio_revenue_to_total', y='Customer_Group_Type', palette='coolwarm')
plt.title('Top 10 Customer Group Types by Revenue-to-Total Ratio', fontsize=16)
plt.xlabel('Revenue-to-Total Ratio', fontsize=12)
plt.ylabel('Customer Group Type', fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
destination_analysis.head(5).plot(
    x='Destination_Country', y='ratio_revenue_to_total', kind='bar', legend=False, title="Top Destination Countries"
)
plt.ylabel('Revenue-to-Total Ratio %')
plt.show()


customer_group_analysis.plot(
    x='Customer_Group_Type', y='ratio_revenue_to_total', kind='bar', legend=False, title="Customer Group Types"
)
plt.ylabel('Revenue-to-Total Ratio %')
plt.show()


In [26]:
canceled_df = orders_file[orders_file['Is_Canceled'] == True].copy()
canceled_df['order_created_at'] = pd.to_datetime(canceled_df['order_created_at'])
canceled_df['MonthYear'] = canceled_df['order_created_at'].dt.to_period('M')

monthly_lost_revenue = canceled_df.groupby('MonthYear')['Revenue_in_usd'].mean().reset_index()
monthly_lost_revenue['MonthYear'] = monthly_lost_revenue['MonthYear'].dt.to_timestamp()


In [88]:
# canceled_df.columns

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(monthly_lost_revenue['MonthYear'], monthly_lost_revenue['Revenue_in_usd'], marker='o', color='red')
plt.title("Average Lost Revenue Due to Cancellations (Monthly)", fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Average Lost Revenue", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [90]:
# orders_file.columns

In [91]:
# orders_file['client_entry_type'].unique()

In [31]:
def group_device(device):
    if device in ['iPad', 'iPod', 'iPhone']:
        return 'Apple Devices'
    elif device in ['Android Phone', 'Android Tablet']:
        return 'Android Devices'
    elif device == 'PC':
        return 'PC'
    elif device == 'BlackBerry':
        return 'BlackBerry'
    elif device in ['Unknown', 'AnonTablet', 'AnonPhone', 'Application']:
        return 'Other'
    else:
        return 'Uncategorized'

In [32]:
orders_file['Device_Group'] = orders_file['Device'].apply(group_device)

In [ ]:
# grouped = orders_file.groupby('Device_Group').agg({
#     'Order_Amount_in_usd': ['mean'], 
#     'Revenue_in_usd': ['mean'], 
#     # 'Device' : 'sum',
#     'client_entry_type': 'sum'
#     # 'Is_Canceled': 'mean',
#     # 'Is_Changed' : 'mean'
# }).reset_index()

# grouped.columns = ['Device_Group', 'Avg_Order_Amount', 'Avg_Revenue',  'entry'] #'Canceled', 'Changed'

# # grouped['Change_Rate'] = (grouped['Changed'] / grouped['Device_count']) * 100
# # grouped['Cancellation_Rate'] = (grouped['Canceled'] / grouped['Device_count']) * 100


# grouped

group_metrics = orders_file.groupby('Customer_Group_Type').agg(
    Total_Orders=('order_id', 'count'),
    Total_Cancellations=('Is_Canceled', 'sum'),
    Total_Changes=('Is_Changed', 'sum')
)


group_metrics['Cancellation_Rate'] = (group_metrics['Total_Cancellations'] / group_metrics['Total_Orders']) * 100
group_metrics['Change_Rate'] = (group_metrics['Total_Changes'] / group_metrics['Total_Orders']) * 100

group_metrics


In [ ]:

sns.set(style="whitegrid")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

sns.barplot(data=group_metrics, x=group_metrics.index, y='Total_Orders', ax=axes[0, 0], palette='Blues_d')
axes[0, 0].set_title('Total Orders by Customer Group Type')
axes[0, 0].set_xlabel('Customer Group Type')
axes[0, 0].set_ylabel('Total Orders')

sns.barplot(data=group_metrics, x=group_metrics.index, y='Total_Cancellations', ax=axes[0, 1], palette='Reds_d')
axes[0, 1].set_title('Total Cancellations by Customer Group Type')
axes[0, 1].set_xlabel('Customer Group Type')
axes[0, 1].set_ylabel('Total Cancellations')

sns.barplot(data=group_metrics, x=group_metrics.index, y='Cancellation_Rate', ax=axes[1, 0], palette='Purples_d')
axes[1, 0].set_title('Cancellation Rate by Customer Group Type')
axes[1, 0].set_xlabel('Customer Group Type')
axes[1, 0].set_ylabel('Cancellation Rate (%)')

sns.barplot(data=group_metrics, x=group_metrics.index, y='Change_Rate', ax=axes[1, 1], palette='Greens_d')
axes[1, 1].set_title('Change Rate by Customer Group Type')
axes[1, 1].set_xlabel('Customer Group Type')
axes[1, 1].set_ylabel('Change Rate (%)')

plt.tight_layout()
plt.show()


In [94]:
booking_system_dist = orders_file.groupby(['Device_Group', 'booking_system_source_type']).size().unstack(fill_value=0)
booking_system_percentage = booking_system_dist.div(booking_system_dist.sum(axis=1), axis=0) * 100

booking_system_percentage

In [95]:
booking_system_dist.plot(kind='bar', stacked=True, figsize=(10, 6))

plt.title('Booking System Distribution by Device Group', fontsize=16)
plt.xlabel('Device Group', fontsize=12)
plt.ylabel('Number of Bookings', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.legend(title='Booking System', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.show()

In [ ]:
group_metrics = orders_file.groupby('Device_Group').agg(
    Total_Orders=('order_id', 'count'),
    Total_Cancellations=('Is_Canceled', 'sum'),
    Total_Changes=('Is_Changed', 'sum')
)


group_metrics['Cancellation_Rate'] = (group_metrics['Total_Cancellations'] / group_metrics['Total_Orders']) * 100
group_metrics['Change_Rate'] = (group_metrics['Total_Changes'] / group_metrics['Total_Orders']) * 100

group_metrics
